### MN

In [1]:
import tensorflow as tf
from collections import namedtuple
from collections import deque
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
_ = plt.gray();

In [2]:
class MNIST(object):
    image = namedtuple("image", "data,normalized_data,w,h,size")
    label = namedtuple("label", "label,encoded,size")
    
    def __init__(self):
        self.train_image = self.read_image("train-images-idx3-ubyte")
        self.test_image = self.read_image("t10k-images-idx3-ubyte")
        self.train_label = self.read_label("train-labels-idx1-ubyte")
        self.test_label = self.read_label("t10k-labels-idx1-ubyte")
    
    def as_int(self, bytes):
        r = 0
        for b in bytes:
            r = (r << 8) | ord(b)
        return r
    
    def read_label(self, filename):
        with open(filename, "rb") as f:
            self.as_int(f.read(4)) # magic number
            size = self.as_int(f.read(4))
            digits = np.array(map(lambda x: ord(x), f.read(-1)))
            encoded = np.zeros([size, 10], dtype=np.float32)
            encoded[np.arange(size),digits] = 1
        return self.label(digits, encoded, size)
    
    def read_image(self, filename):
        with open(filename, "rb") as f:
            self.as_int(f.read(4)) # magic number
            size = self.as_int(f.read(4))
            w = self.as_int(f.read(4))
            h = self.as_int(f.read(4))
            data = np.array(map(lambda x: ord(x), f.read(-1)), dtype=np.float32).reshape(-1, w*h)
            norm = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-5)
        return self.image(data=data, normalized_data=norm, w=w, h=h, size=size)
    
    def _range_max(self, row, length):
        queue = deque()
        output = [0 for _ in xrange(len(row) - length + 1)]
        length -= 1
        for j, e in enumerate(row):
            while(len(queue) != 0 and queue[0][0] <= e):
                queue.popleft()
            queue.appendleft((e, j))
            if j < length:
                continue
            while(queue[-1][1] < (j - length)):
                queue.pop()
            
            output[j - length] = queue[-1][0]
            
        return output
        
    def _grid_max(self, arr2d, grid=(4, 4)):
        temp = np.array([self._range_max(arr1d, length=grid[0]) for arr1d in arr2d]).T
        return np.array([self._range_max(arr1d, length=grid[1]) for arr1d in temp]).T
    
    def augment_feature(self, data, grid=(5, 5)):
        """
        data is a 784xS size 2d array
        each row is reshaped 28x28 image
        lets calculate max in each grid
        """
        imsize = 28
        data_cube = data.reshape(data.shape[0], imsize, imsize)
        return np.array([self._grid_max(matrix, grid) for matrix in data_cube])

mnist = MNIST()

In [3]:
def next_batch(self, size):
    select = np.random.choice(self.train_image.size, size)
    return (self.train_image.normalized_data[select,:], self.train_label.encoded[select,:])

In [38]:
tf.reset_default_graph()

def variable_summaries(var, name):
    with tf.name_scope('summaries/' + name):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

x = tf.placeholder(tf.float64, [None, 784])
keep_prob = tf.placeholder(dtype=tf.float64)
dropped = tf.nn.dropout(x, keep_prob)

with tf.variable_scope("Layer-1"):
    W1 = tf.get_variable("W1", shape=[784, 400], dtype=tf.float64, initializer=tf.glorot_uniform_initializer())
    b1 = tf.get_variable("b1", shape=[400], dtype=tf.float64, initializer=tf.zeros_initializer())
    y1 = tf.matmul(dropped, W1) + b1
    variable_summaries(W1, "Weights")
    variable_summaries(b1, "Biases")

with tf.variable_scope("Layer-1-Activation"):
    y1_ = tf.nn.relu(y1)
    variable_summaries(y1_, "Relu-Activations")

with tf.variable_scope("Layer-2"):
    W = tf.get_variable("W2", shape=[400, 10], dtype=tf.float64, initializer=tf.glorot_uniform_initializer())
    b = tf.Variable(tf.zeros([10], dtype=tf.float64), name="bias")
    y = tf.matmul(y1_, W) + b
    variable_summaries(W, "Weights")
    variable_summaries(b, "Biases")
    variable_summaries(y, "WX_plus_b")
    y_ = tf.placeholder(tf.float32, [None, 10])

with tf.variable_scope("Loss"):
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y), name="Cross-Entropy-Loss")

train_step = tf.train.GradientDescentOptimizer(0.02).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('train', sess.graph)
    tf.global_variables_initializer().run()
    # Train
    batch_size = 100
    iterations = 20000
    for i in range(iterations):
        batch_xs, batch_ys = next_batch(mnist, batch_size)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.7})
        
        if (batch_size * i % 10000) == 0:
            batch_xs, batch_ys = mnist.test_image.normalized_data, mnist.test_label.encoded
            summary, acc_, loss_ = sess.run([merged, accuracy, cross_entropy], 
                                            feed_dict={x: batch_xs, y_: batch_ys, keep_prob:1.0})
            train_writer.add_summary(summary, global_step=i * batch_size / 10000)
            print "accuracy: %0.4f, loss=%0.6f" % (acc_, loss_)
        
    # Test trained model
    print(sess.run(accuracy, feed_dict={x: mnist.test_image.normalized_data,
                                      y_: mnist.test_label.encoded, keep_prob:1.0}))

accuracy: 0.1030, loss=2.567219
accuracy: 0.8519, loss=0.539366
accuracy: 0.8853, loss=0.410318
accuracy: 0.8988, loss=0.349932
accuracy: 0.9113, loss=0.316727
accuracy: 0.9149, loss=0.296560
accuracy: 0.9187, loss=0.283441
accuracy: 0.9193, loss=0.274365
accuracy: 0.9256, loss=0.261219
accuracy: 0.9268, loss=0.249327
accuracy: 0.9302, loss=0.238319
accuracy: 0.9306, loss=0.234933
accuracy: 0.9339, loss=0.229801
accuracy: 0.9329, loss=0.224741
accuracy: 0.9376, loss=0.217748
accuracy: 0.9394, loss=0.211483
accuracy: 0.9408, loss=0.205580
accuracy: 0.9406, loss=0.202732
accuracy: 0.9427, loss=0.199952
accuracy: 0.9427, loss=0.197214
accuracy: 0.9438, loss=0.192560
accuracy: 0.9453, loss=0.188550
accuracy: 0.9467, loss=0.183512
accuracy: 0.9483, loss=0.182057
accuracy: 0.9481, loss=0.180020
accuracy: 0.9482, loss=0.178750
accuracy: 0.9494, loss=0.175059
accuracy: 0.9504, loss=0.172079
accuracy: 0.9507, loss=0.168491
accuracy: 0.9525, loss=0.167392
accuracy: 0.9523, loss=0.165757
accuracy